In [1]:
import requests
import csv
import os
import pandas as pd
import geojson
import json
import geopandas as gpd
from shapely.geometry import Point
from sqlalchemy import create_engine

In [ ]:
# This same block of code were used for both battlefield and historical site API searches
# Define the API key and base URLs
API_KEY = 'API Key'  # API key removed for privacy purpose
endpoint_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
details_url = 'https://maps.googleapis.com/maps/api/place/details/json'

# Read coordinates from CSV file
input_csv = 'Data\csv\county_pts.csv'  # CSV file with center of Kansas county Lat_Y and Long_X coordinates

# Initialize a counter for output files
file_counter = 1

# Open input CSV and iterate through each row for Lat_Y and Long_X
with open(input_csv, mode='r', newline='', encoding='utf-8') as input_file:
    csv_reader = csv.DictReader(input_file)
    
    for row in csv_reader:
        latitude = row['Lat_Y']
        longitude = row['Long_X']
        location = f"{latitude},{longitude}"

        # Define the output file name based on the counter
        output_folder = "Data/csv/Battlefield_API_results"
        output_csv = os.path.join(output_folder, f'Battlefield{file_counter}.csv') # Replaced file name "Battlefield" with "Historical"
        
        # Define parameters for the API request using coordinates from the CSV file
        params = {
            'location': location,  # Coordinates from the CSV
            'radius': '50000',  # 50,000 meters is max allowed search radius
            'keyword': 'Battlefield',  # Search keyword also include 'Historical' as a search keyword for historical sites
            'key': API_KEY
        }

        # Send the request to the Places API
        response = requests.get(endpoint_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            results = response.json().get('results', [])
            if results:
                # Open CSV file for output
                with open(output_csv, mode='w', newline='', encoding='utf-8') as output_file:
                    writer = csv.writer(output_file)
                    # Write header row to the CSV file, including City, Type, and Description
                    writer.writerow(['Place Name', 'Address', 'City', 'Rating', 'Total Ratings', 'Latitude', 'Longitude', 'Type', 'Description'])

                    # Iterate through each place result
                    for place in results:
                        name = place.get('name', 'No name available')
                        formatted_address = place.get('vicinity', 'No address available')
                        rating = place.get('rating', 'No rating available')
                        total_ratings = place.get('user_ratings_total', 'No ratings available')
                        lat = place['geometry']['location'].get('lat', 'No latitude available')
                        lng = place['geometry']['location'].get('lng', 'No longitude available')
                        types = ', '.join(place.get('types', ['No types available']))

                        # Split the formatted_address to extract City
                        formatted_address_parts = formatted_address.split(', ')
                        city = formatted_address_parts[-1] if len(formatted_address_parts) >= 1 else 'No city available'

                        # Get place details if available
                        place_id = place.get('place_id', None)
                        description = 'No description available'
                        
                        if place_id:
                            details_params = {
                                'place_id': place_id,
                                'fields': 'name,formatted_address,geometry,place_id,rating,user_ratings_total,reviews,types',
                                'key': API_KEY
                            }
                            details_response = requests.get(details_url, params=details_params)

                            # Extract description from the reviews if available
                            if details_response.status_code == 200:
                                details_result = details_response.json().get('result', {})
                                if 'reviews' in details_result and details_result['reviews']:
                                    description = details_result['reviews'][0].get('text', 'No description available')

                        # Print place details 
                        print(f"Place Name: {name}")
                        print(f"Address: {formatted_address}")
                        print(f"City: {city}")
                        print(f"Rating: {rating}")
                        print(f"Total Ratings: {total_ratings}")
                        print(f"Latitude: {lat}")
                        print(f"Longitude: {lng}")
                        print(f"Type: {types}")
                        print(f"Description: {description}\n")

                        # Write place details to the CSV file
                        writer.writerow([name, formatted_address, city, rating, total_ratings, lat, lng, types, description])

                # Increment the file counter for the next output file
                file_counter += 1
            else:
                print(f"No results found for location: {location}")
        else:
            print(f"Error: Unable to fetch places data for location {location}. Status Code: {response.status_code}")
            print(f"Response: {response.text}")

In [6]:
# Specify the folder containing Battlefield CSV files
folder_path = 'Data/csv/Battlefield_API_results'

# Create an empty list to hold DataFrames
df_list = []

# Loop through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.csv'):  # Ensure only CSV files are processed
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)  # Read each CSV file into a DataFrame
        df_list.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(df_list, ignore_index=True)

# Show the merged DataFrame
print(merged_df)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('Data/csv/Battlefield_API_results/merged_battlefields.csv', index=False)

                                           Place Name  \
0   Battle of Punished Woman's Fork National Histo...   
1                      Nerf Warz Battlefield & Arcade   
2   Battle of Punished Woman's Fork National Histo...   
3                              Black Jack Battlefield   
4                                          Fort Titus   
..                                                ...   
67                Battle of Carthage Civil War Museum   
68                           Battle Of Beecher Island   
69                                     Beecher Island   
70                                     Armel Cemetery   
71                                         Fort Titus   

                          Address        City               Rating  \
0               67871, Scott City  Scott City                  4.8   
1      6821 E Kellogg Dr, Wichita     Wichita                  3.8   
2               67871, Scott City  Scott City                  4.8   
3       163 E 2000 Rd, Wellsville  

In [2]:
# Specify the folder containing Historical CSV files
folder_path = 'Data/csv/Historical_Sites_API_results'

# Create an empty list to hold DataFrames
df_list = []

# Loop through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.csv'):  # Ensure only CSV files are processed
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)  # Read each CSV file into a DataFrame
        df_list.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
merged_hist_df = pd.concat(df_list, ignore_index=True)

# Show the merged DataFrame
print(merged_hist_df)

# Optionally, save the merged DataFrame to a new CSV file
merged_hist_df.to_csv('Data/csv/Historical_Sites_API_results/merged_historical.csv', index=False)

                                            Place Name  \
0                  Greenwood County Historical Society   
1                                           Teter Rock   
2                      Historic Frisco Railroad Bridge   
3                      Coffey County Historical Museum   
4                                   Hartford Calaboose   
...                                                ...   
2040                           Fromm Stone Arch Bridge   
2041                    Kansas' Oldest Swinging Bridge   
2042  Caney Valley Historical Society & Museum Complex   
2043                        South School historic site   
2044                    Riverside Park Dinosaur Statue   

                          Address          City  Rating  Total Ratings  \
0            120 W 4th St, Eureka        Eureka     5.0              2   
1              2555 300th, Eureka        Eureka     4.8            173   
2               M98V+33, Keighley      Keighley     5.0              2   
3      

In [3]:
# Load battlefield csv to dataframe
battlefields_df = pd.read_csv('Data/csv/KS_battlefields.csv', encoding='utf-8')

# Load historical sites csv file with 'Latin1' encoding (this is done because utf-8 didn't work)
historical_df = pd.read_csv('Data/csv/KS_Historical.csv', encoding='latin1')

# Save the DataFrame to a new csv file with 'utf-8' encoding
historical_df.to_csv('Data/csv/KS_Historical_utf8.csv', index=False, encoding='utf-8')

#Verify the new file encoding format
historical_utf8_df = pd.read_csv('Data/csv/KS_Historical_utf8.csv', encoding='utf-8')
print(historical_utf8_df)

                                            Place_Name  \
0                  Greenwood County Historical Society   
1                                           Teter Rock   
2                      Historic Frisco Railroad Bridge   
3                      Coffey County Historical Museum   
4                                   Hartford Calaboose   
...                                                ...   
1789                      The Old Jefferson Mercantile   
1790                           Fromm Stone Arch Bridge   
1791                    Kansas' Oldest Swinging Bridge   
1792  Caney Valley Historical Society & Museum Complex   
1793                    Riverside Park Dinosaur Statue   

                                Address          City  Rating  Total_Rati  \
0                  120 W 4th St, Eureka        Eureka     5.0           2   
1                    2555 300th, Eureka        Eureka     4.8         173   
2                     M98V+33, Keighley      Keighley     5.0           

In [4]:
# Define columns to drop for historical sites
columns_to_remove = ['Address', 'Rating', 'Total_Rati', 'Type', 'Descriptio']  

# Drop the specified columns
historical_utf8_df = historical_utf8_df.drop(columns=columns_to_remove)
historical_utf8_df.head(5)

,Place_Name,City,Latitude,Longitude
0,Greenwood County Historical Society,Eureka,37.822396,-96.295194
1,Teter Rock,Eureka,38.029563,-96.423564
2,Historic Frisco Railroad Bridge,Keighley,37.665213,-96.607365
3,Coffey County Historical Museum,Burlington,38.194431,-95.749326
4,Hartford Calaboose,Hartford,38.308111,-95.957775


In [5]:
# Define columns to drop for battlefields
columns_to_remove = ['Rating', 'Total_Rati', 'Type', 'Descriptio']  

# Drop the specified columns
battlefields_df = battlefields_df.drop(columns=columns_to_remove)
battlefields_df.head(5)

,Place_Name,City,Latitude,Longitude
0,Battle of Punished Woman's Fork National Histo...,Scott City,38.643554,-100.927941
1,Battlefield Cemetery,Pleasanton,38.122166,-94.723388
2,Black Jack Battlefield,Wellsville,38.762218,-95.132963
3,Battle of Fort Titus,Lecompton,39.044316,-95.391331
4,Mine Creek Civil War Battlefield State Histori...,Pleasanton,38.145021,-94.723436


In [7]:
# Merge Kansas Battlefield and Historical Sites into one data file
merged_df = pd.merge(battlefields_df, historical_utf8_df, on=['Place_Name', 'City', 'Latitude', 'Longitude'], how='outer', suffixes=('_battlefields', '_historical'))
merged_df['State'] = 'Kansas'

print(merged_df.head())

merged_df.to_csv('Data/csv/merged_battlefields_historical.csv', index=False)

                            Place_Name        City   Latitude   Longitude  \
0  "Four Horsemen of the Lines" Marker      Horton  39.671102  -95.527050   
1  "Four Horsemen of the Lines" Marker      Horton  39.671102  -95.527050   
2                    1139 Steam Engine  Dodge City  37.753021 -100.022025   
3                    1139 Steam Engine  Dodge City  37.753021 -100.022025   
4                    1139 Steam Engine  Dodge City  37.753021 -100.022025   

    State  
0  Kansas  
1  Kansas  
2  Kansas  
3  Kansas  
4  Kansas  


In [4]:
# Postgres database credentials
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"  
database = "Project3"

# Create connection 
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

In [3]:
# Test the connection by executing a query
try:
    # Run a test query; replace 
    test_query = "SELECT * FROM cemetery LIMIT 5;"
    test_df = pd.read_sql_query(test_query, con=engine)
    print("Connection successful. Sample data:")
    display(test_df)
except Exception as e:
    print("Connection failed. Error:")
    print(e)

Connection successful. Sample data:


,name,city,longitude,latitude
0,Goldamer Cemetery,Dodge City,-100.907233,37.772977
1,Haskell Cemetery,Sublette,-100.875721,37.554617
2,Buda Cemetery,Ness City,-100.175850,38.289848
3,Old Mission Cemetery,Wichita,-97.296711,37.724458
4,Fairview Cemetery,Cheney,-97.786108,37.619584


In [11]:
# Define the query to select specific columns for cemetery
query = "SELECT name, city, longitude, latitude FROM cemetery;"

# Run the query and store the result in a DataFrame
cemetery_df = pd.read_sql_query(query, con=engine)

# Display the DataFrame to inspect the data
cemetery_df.head(5)


,name,city,longitude,latitude
0,Goldamer Cemetery,Dodge City,-100.907233,37.772977
1,Haskell Cemetery,Sublette,-100.875721,37.554617
2,Buda Cemetery,Ness City,-100.175850,38.289848
3,Old Mission Cemetery,Wichita,-97.296711,37.724458
4,Fairview Cemetery,Cheney,-97.786108,37.619584


In [5]:
# Export cemetery to GeoJSON format
cemetery_df['geometry'] = cemetery_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
cemetery_gdf = gpd.GeoDataFrame(cemetery_df, geometry='geometry')
cemetery_gdf.to_file("Data\geojson\cemetery.geojson", driver="GeoJSON")


In [6]:
# Load and read the GeoJSON file for verification
geojson_file_path = 'Data\geojson\cemetery.geojson' 

with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Display the GeoJSON content using pretty print
print(json.dumps(geojson_data, indent=2))  


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "name": "Goldamer Cemetery",
        "city": "Dodge City",
        "longitude": -100.907233,
        "latitude": 37.772977
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -100.907233,
          37.772977
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Haskell Cemetery",
        "city": "Sublette",
        "longitude": -100.875721,
        "latitude": 37.554617
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -100.875721,
          37.554617
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Buda Cemetery",
        "city": "Ness City",
        "longitude": -100.17585,
        "latitude": 38.289848
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -100.17585,
          38.

In [10]:
# Define the query to select specific columns for haunted places
query = "SELECT city, description, location_name, state, longitude, latitude FROM haunted_places;"

# Run the query and store the result in a DataFrame
haunted_places_df = pd.read_sql_query(query, con=engine)

# Display the DataFrame to inspect the data
haunted_places_df.head(5)

,city,description,location_name,state,longitude,latitude
0,Alma,The Alma Cemetery is haunted for two reasons. ...,Alma Cemetery,Kansas,-96.299018,39.016088
1,Alma,Wabaunsee High School is the Haunted by past s...,Wabaunsee High School,Kansas,-96.289219,39.018726
2,Arcadia,When in this church day or night you can see s...,the church of Gods word,Kansas,-95.468087,37.667773
3,Atchison,The local newspaper office's are said to be ha...,The Globe,Kansas,-95.116777,39.561630
4,Atchison,It is said that on the stroke of midnight the ...,Jackson Park,Kansas,-95.117802,39.543923


In [8]:
# Export haunted places to GeoJSON format
haunted_places_df['geometry'] = haunted_places_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
haunted_places_df = gpd.GeoDataFrame(haunted_places_df, geometry='geometry')
haunted_places_df.to_file("Data\geojson\haunted_places.geojson", driver="GeoJSON")

In [9]:
# Load and read the GeoJSON file for verification
geojson_file_path = 'Data\geojson\haunted_places.geojson' 

with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Display the GeoJSON content using pretty print
print(json.dumps(geojson_data, indent=2))  


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "city": "Alma",
        "description": "The Alma Cemetery is haunted for two reasons. One is the people who are buried there and refuse to leave. Some pictures have been taken during the night up there. (It's the best time.) The pictures were unbeleivable, there were orbs and mist figures in t",
        "location_name": "Alma Cemetery",
        "state": "Kansas                                  ",
        "longitude": -96.2990179,
        "latitude": 39.016088
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -96.2990179,
          39.016088
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "city": "Alma",
        "description": "Wabaunsee High School is the Haunted by past students and teachers. Most of the students groups are those of students who never got to graduate from the high school because they 

In [5]:
# Define the query to select specific columns for historical sites
query = "SELECT place_name, city, longitude, latitude, state FROM historical_sites;"

# Run the query and store the result in a DataFrame
historical_df = pd.read_sql_query(query, con=engine)

# Display the DataFrame to inspect the data
historical_df.head(5)

,place_name,city,longitude,latitude,state
0,Four Horsemen of the Lines Marker,Horton,39.671102,-95.527050,Kansas
1,Four Horsemen of the Lines Marker,Horton,39.671102,-95.527050,Kansas
2,1139 Steam Engine,Dodge City,37.753021,-100.022025,Kansas
3,1139 Steam Engine,Dodge City,37.753021,-100.022025,Kansas
4,1139 Steam Engine,Dodge City,37.753021,-100.022025,Kansas


In [6]:
# Export historical sites to GeoJSON format
historical_df['geometry'] = historical_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
historical_df = gpd.GeoDataFrame(historical_df, geometry='geometry')
historical_df.to_file("Data\geojson\historical_sites.geojson", driver="GeoJSON")

In [7]:
# Load and read the GeoJSON file for verification
geojson_file_path = 'Data\geojson\historical_sites.geojson' 

with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Display the GeoJSON content using pretty print
print(json.dumps(geojson_data, indent=2))  


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "place_name": "Four Horsemen of the Lines Marker",
        "city": "Horton",
        "longitude": 39.6711021,
        "latitude": -95.5270499,
        "state": "Kansas                                  "
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          39.6711021,
          -95.5270499
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "place_name": "Four Horsemen of the Lines Marker",
        "city": "Horton",
        "longitude": 39.6711021,
        "latitude": -95.5270499,
        "state": "Kansas                                  "
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          39.6711021,
          -95.5270499
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "place_name": "1139 Steam Engine",
        "city": "Dodge City",
 

In [8]:
# Define the query to select specific columns for medical locations
query = "SELECT name, city, state, long, lat FROM medical;"

# Run the query and store the result in a DataFrame
medical_df = pd.read_sql_query(query, con=engine)

# Display the DataFrame to inspect the data
medical_df.head(5)

,name,city,state,long,lat
0,Kiowa District Hospital,Kiowa,KS,-98.483599,37.015142
1,Kansas Medical Center,Andover,KS,-97.147354,37.724361
2,Wamego Health Center,Wamego,KS,-96.319086,39.204919
3,South Central Kansas Medical Center,Arkansas City,KS,-97.038642,37.133045
4,Dwight D Eisenhower Veterans Affairs Medical C...,Leavenworth,KS,-94.896593,39.282413


In [9]:
# Export medical data to GeoJSON format
medical_df['geometry'] = medical_df.apply(lambda row: Point(row['long'], row['lat']), axis=1)
medical_df = gpd.GeoDataFrame(medical_df, geometry='geometry')
medical_df.to_file("Data\geojson\medical.geojson", driver="GeoJSON")

In [10]:
# Load and read the GeoJSON file for verification
geojson_file_path = 'Data\geojson\medical.geojson' 

with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Display the GeoJSON content using pretty print
print(json.dumps(geojson_data, indent=2))  

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "name": "Kiowa District Hospital",
        "city": "Kiowa",
        "state": "KS",
        "long": -98.4835989998,
        "lat": 37.0151420001
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -98.4835989998,
          37.0151420001
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Kansas Medical Center",
        "city": "Andover",
        "state": "KS",
        "long": -97.1473541263,
        "lat": 37.7243614195
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -97.1473541263,
          37.7243614195
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Wamego Health Center",
        "city": "Wamego",
        "state": "KS",
        "long": -96.3190860268,
        "lat": 39.2049189586
      },
      "geometry": {
 

In [13]:
# Define the query to select specific columns for medical locations
query = "SELECT state, county, population FROM us_census;"

# Run the query and store the result in a DataFrame
census_df = pd.read_sql_query(query, con=engine)

# Display the DataFrame to inspect the data
census_df.head(5)

,state,county,population
0,Alabama,Autauga County,58805
1,Alabama,Baldwin County,231767
2,Alabama,Barbour County,25223
3,Alabama,Bibb County,22293
4,Alabama,Blount County,59134


In [15]:
# Convert the Census dataframe to json file
census_df.to_json('Data/geojson/us_census_data_2024.json', orient='records', lines=True)